In [5]:
import json
import pandas as pd

league_id = '008da363-45af-4a07-87d2-f22567e27213'

from src.get_summoner_ids_from_league_id import get_summoner_ids_from_league_id
from src.get_puuid_by_summon_id import get_puuid_by_summon_id
from src.get_match_ids_by_puuid import get_match_ids_by_puuid
from src.get_match_timeline import get_match_timeline
from src.get_match_by_match_id import get_match_by_match_id
from src.extract_events_from_timeline import extract_events_from_timeline

In [10]:
pd.DataFrame(get_summoner_ids_from_league_id(league_id)).to_csv('tests/files/summoner_ids_from_league_id',index=False,header=False)

In [61]:
import os
import re

def get_match_ids_from_files(folder="match_timelines"):
    # List all filenames in the directory
    filenames = os.listdir(folder)

    # Prepare an empty list to store the match IDs
    selected_match_ids_list = []

    # Go through each filename
    for filename in filenames:
        # Use regex to match the pattern 'NA1_[0-9]+.json' and extract the match ID
        match_id = re.search('(NA1_[0-9]+).json', filename)
        # If a match ID was found, add it to the list
        if match_id:
            selected_match_ids_list.append(match_id.group(1))

    return selected_match_ids_list

# Specify the directory you want to scan
folder = "match_timelines"
selected_match_ids_list = get_match_ids_from_files(folder)
print(selected_match_ids_list)


['NA1_4334085229', 'NA1_3720451304', 'NA1_3999408652', 'NA1_4730272790', 'NA1_4729028871', 'NA1_4316964052', 'NA1_4144801485', 'NA1_4730458896', 'NA1_4729236700', 'NA1_4732725106', 'NA1_4731598809', 'NA1_4317000585', 'NA1_4733176682', 'NA1_4661776183', 'NA1_4732158712', 'NA1_3721689378', 'NA1_4733221817', 'NA1_4729288774', 'NA1_4716426808', 'NA1_4731630308', 'NA1_4732075515', 'NA1_4732635752', 'NA1_4731700781', 'NA1_4732269793', 'NA1_4729253603', 'NA1_3951298981', 'NA1_4722530575', 'NA1_4722510115', 'NA1_4730426211', 'NA1_4731725908', 'NA1_4582748967', 'NA1_4586574877', 'NA1_4010264452', 'NA1_4594678019', 'NA1_4733271978', 'NA1_4114138238', 'NA1_4579507558', 'NA1_4732316254', 'NA1_4578359753', 'NA1_4733191591', 'NA1_4733050255', 'NA1_4733014869', 'NA1_4526318966', 'NA1_4586671320', 'NA1_4620394242', 'NA1_3721387703', 'NA1_4722522985', 'NA1_4661792899', 'NA1_4731564720', 'NA1_4582928739', 'NA1_4729854000', 'NA1_4732342030', 'NA1_4111864936', 'NA1_4426361344', 'NA1_4573998043', 'NA1_3985

In [64]:
import json
import os
from rate_limiter import rl_sec, rl_min

# Create the 'matches' folder if it doesn't exist
folder = 'matches'
if not os.path.exists(folder):
    os.makedirs(folder)

# Get match 
match_dict = {}
for match_id in selected_match_ids_list:
    # Apply rate limiting
    rl_sec()
    rl_min()

    # Get match data
    match = get_match_by_match_id(match_id)

    # Store the match JSONs
    match_dict[match_id] = match

    # Save the JSON data to a file inside the 'matches' folder
    with open(os.path.join(folder, f'{match_id}.json'), 'w') as json_file:
        json.dump(match, json_file)

In [80]:
import os
import json
import pandas as pd

folder = 'matches'
filenames = os.listdir(folder)

dataframes = []  # to store all dataframes
match_dict = {}
for filename in filenames:
    if filename.endswith('.json'):  # to ensure we're working with JSON files
        with open(os.path.join(folder, filename), 'r') as json_file:
            data = json.load(json_file)
            # ignore .json at end of filename
            match_dict[filename[:-5]] = (data['metadata'], data['info'])

In [97]:
import pandas as pd

# Assuming match_dict is a dictionary containing data related to matches
# Accessing the DataFrame and extracting its columns
columns_list = list(pd.DataFrame(match_dict['NA1_4334085229'][1]['participants']).columns)

# Creating a DataFrame from the list of columns
columns_df = pd.DataFrame(columns_list, columns=['Column Names'])

# Save the DataFrame as a CSV file
columns_df.to_csv('fields.csv', index=False)


# Prep participant data ready for upload

# Connect to postgreSQL

In [2]:
from sqlalchemy import text
from sqlalchemy.exc import OperationalError
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

def read_txt(filename):
    with open(filename, 'r') as file:
        return file.read().strip()

password = read_txt('postgres_pw.txt')
host = 'localhost'
database_name = 'loldb'
port = '5432'
engine = create_engine(f'postgresql://postgres:{password}@{host}:{port}/{database_name}')

# Try connecting and executing a simple query
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Successfully connected!")
except OperationalError as e:
    print(f"Connection failed: {e}")

Successfully connected!


# Load match data into SQL, for all match.json files

In [5]:
included_match_fields = pd.read_csv('included_match_fields.csv')
included_fields_list = included_match_fields.iloc[:, 0].tolist()

df = pd.DataFrame(match[1]['participants'])[included_fields_list]
df

NameError: name 'match' is not defined

In [ ]:
def extract_match_data(player_data):
    included_fields = pd.read_csv('included_match_fields.csv')
    included_fields_list = included_fields.iloc[:, 0].tolist()
    
    metadata = player_data[0]
    match_id = metadata['matchId']
    
    info = player_data[1]
    
    data = pd.DataFrame(info['participants'])[included_fields_list]
    data.insert(0, 'PUUid', metadata['participants'])
    data.insert(0, 'match_id', match_id)
    
    return data

In [286]:
from extract_player_data_from_match import extract_player_data_from_match

included_fields = pd.read_csv('included_match_fields.csv')
included_fields_list = included_fields.iloc[:, 0].tolist()

for match_id, match in match_dict.items():
    PUUids = match[0]['participants']
    if len(PUUids) == 10:
        selected_data = pd.DataFrame(match[1]['participants'])[included_fields_list]
        selected_data.insert(0, 'PUUid', PUUids)
        selected_data.insert(0, 'match_id', match_id)
        
        selected_data.to_sql('player_match_data', engine, if_exists='append', index=False)
    else:
        print(f"Match {match_id} has fewer than 10 participants and won't be added to the database.")

# TODO handle partially full matches

Match NA1_4730272790 has fewer than 10 participants and won't be added to the database.
Match NA1_4730458896 has fewer than 10 participants and won't be added to the database.
Match NA1_4731598809 has fewer than 10 participants and won't be added to the database.
Match NA1_4731630308 has fewer than 10 participants and won't be added to the database.
Match NA1_4731700781 has fewer than 10 participants and won't be added to the database.
Match NA1_4732269793 has fewer than 10 participants and won't be added to the database.
Match NA1_4722530575 has fewer than 10 participants and won't be added to the database.
Match NA1_4722510115 has fewer than 10 participants and won't be added to the database.
Match NA1_4730426211 has fewer than 10 participants and won't be added to the database.
Match NA1_4731725908 has fewer than 10 participants and won't be added to the database.
Match NA1_4732316254 has fewer than 10 participants and won't be added to the database.
Match NA1_4722522985 has fewer t

# Load extract_match_data(match_dict[match_id] to SQL

In [275]:
# from sqlalchemy import text, create_engine
# from sqlalchemy.exc import OperationalError

# # Assuming 'engine' is your database engine
# try:
#     with engine.connect() as connection:
#         connection.execute(text("DROP TABLE IF EXISTS player_match_data"))
#         print("Table 'player_match_data' dropped successfully.")
# except OperationalError as e:
#     print(f"Operation failed: {e}")

Table 'player_match_data' dropped successfully.


In [6]:
try:
    # Fetch data from the table and print it
    with engine.connect() as connection:
        result = connection.execute(text('SELECT count(distinct "PUUid") FROM player_match_data'))
        data_df = pd.DataFrame(result.fetchall(), columns=result.keys())
        print(data_df)
except OperationalError as e:
    print(f"Connection failed: {e}")

   count
0   1204


In [8]:
# Query the db

try:
    # Fetch data from the table and print it
    with engine.connect() as connection:
        result = connection.execute(text('SELECT count(distinct "championName") FROM player_match_data'))
        data_df = pd.DataFrame(result.fetchall(), columns=result.keys())
        print(data_df)
except OperationalError as e:
    print(f"Connection failed: {e}")


   count
0    162


In [303]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

def read_txt(file_path):
    with open(file_path, 'r') as f:
        return f.read().strip()

app = Flask(__name__)

# Update this to your database credentials
password = read_txt('postgres_pw.txt')
host = 'localhost'
database_name = 'loldb'
port = '5432'

app.config['SQLALCHEMY_DATABASE_URI'] = f'postgresql://postgres:{password}@{host}:{port}/{database_name}'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)

    def __repr__(self):
        return '<User %r>' % self.username


In [305]:
with app.app_context():
    db.create_all()

with app.app_context():
    new_user = User(username='nolan', email='meyer09@gmail.com')
    db.session.add(new_user)
    db.session.commit()

In [310]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
# Update this to your database credentials
password = read_txt('postgres_pw.txt')
host = 'localhost'
database_name = 'loldb'
port = '5432'

app.config['SQLALCHEMY_DATABASE_URI'] = f'postgresql://postgres:{password}@{host}:{port}/{database_name}'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

class Match(db.Model):
    __tablename__ = 'player_match_data'

    id = db.Column(db.Integer, primary_key=True)
    # other fields...

@app.route('/get_match', methods=['GET'])
def get_match():
    match_id = request.args.get('match_id', default = 1, type = int)
    match = Match.query.get(match_id)
    if match is None:
        return jsonify({'error': 'Match not found'}), 404
    else:
        return jsonify({'match_id': match.id})  # modify this line to return required fields